# Demo TCP server and Client in Go


```go
package main

import (
	"fmt"
	"log"
	"net"
	"time"
)

func main() {

	// listen on a port
	ln, err := net.Listen("tcp", ":8080")
	defer ln.Close()
	if err != nil {
		log.Panic(err)
	}

	for {
		// accept a connection
		c, err := ln.Accept()
		if err != nil {
			log.Fatal(err)
			continue
		}

		// handle the connection
		go func(conn net.Conn) {

			daytime := time.Now().String() + " ...Hallo\n"
			// conn.Write([]byte(daytime))
			fmt.Fprintf(conn, daytime)
			conn.Close()
		}(c)
	}
}
```


A TCP client written in Go could look like in the following:

```go
package main

import (
	"bufio"
	"fmt"
	"log"
	"net"
)

func main() {
	conn, err := net.Dial("tcp", "192.168.20.2:8080")
	defer conn.Close()
	if err != nil {
		log.Panic(err)
	}

	connbuf := bufio.NewReader(conn)
	for {
		str, err := connbuf.ReadString('\n')
		if len(str) > 0 {
			fmt.Println(str)
            break
		}
		if err != nil {
			break
		}
	}
}
```

# Intro to TCPDump


The manual `man tcpdum` says:

  > Tcpdump  prints out a description of the contents of packets on a network interface...
  
Go ahead and have a look at it.

```bash
TCPDUMP(8)                      System Manager's Manual                      TCPDUMP(8)

NAME
       tcpdump - dump traffic on a network

SYNOPSIS
       tcpdump [ -AbdDefhHIJKlLnNOpqStuUvxX# ] [ -B buffer_size ]
               [ -c count ]
               [ -C file_size ] [ -G rotate_seconds ] [ -F file ]
               [ -i interface ] [ -j tstamp_type ] [ -m module ] [ -M secret ]
               [ --number ] [ -Q in|out|inout ]
               [ -r file ] [ -V file ] [ -s snaplen ] [ -T type ] [ -w file ]
               [ -W filecount ]
               [ -E spi@ipaddr algo:secret,...  ]
               [ -y datalinktype ] [ -z postrotate-command ] [ -Z user ]
               [ --time-stamp-precision=tstamp_precision ]
               [ --immediate-mode ] [ --version ]
               [ expression ]

DESCRIPTION
       Tcpdump  prints out a description of the contents of packets on a network inter‐
       face that match the boolean expression; the description is preceded  by  a  time
       stamp,  printed, by default, as hours, minutes, seconds, and fractions of a sec‐
       ond since midnight.  It can also be run with the -w flag,  which  causes  it  to
       save  the  packet  data  to  a file for later analysis, and/or with the -r flag,
       which causes it to read from a saved packet file rather  than  to  read  packets
       from  a  network  interface  (please  note tcpdump is protected via an enforcing
       apparmor(7) profile in Ubuntu which limits the files tcpdump  may  access).   It
       can also be run with the -V flag, which causes it to read a list of saved packet
       files. In all cases, only packets that match expression  will  be  processed  by
       tcpdump.

       Tcpdump  will,  if not run with the -c flag, continue capturing packets until it
       is interrupted by a SIGINT signal (generated, for example, by typing your inter‐
       rupt  character,  typically  control-C) or a SIGTERM signal (typically generated
       with the kill(1) command); if run with the -c  flag,  it  will  capture  packets
       until it is interrupted by a SIGINT or SIGTERM signal or the specified number of
       packets have been processed.
```

In [1]:
%%bash

man tcpdump

TCPDUMP(8)                  System Manager's Manual                 TCPDUMP(8)

NAME
       tcpdump - dump traffic on a network

SYNOPSIS
       tcpdump [ -AbdDefhHIJKlLnNOpqStuUvxX# ] [ -B buffer_size ]
               [ -c count ]
               [ -C file_size ] [ -G rotate_seconds ] [ -F file ]
               [ -i interface ] [ -j tstamp_type ] [ -m module ] [ -M secret ]
               [ --number ] [ -Q in|out|inout ]
               [ -r file ] [ -V file ] [ -s snaplen ] [ -T type ] [ -w file ]
               [ -W filecount ]
               [ -E spi@ipaddr algo:secret,...  ]
               [ -y datalinktype ] [ -z postrotate-command ] [ -Z user ]
               [ --time-stamp-precision=tstamp_precision ]
               [ --immediate-mode ] [ --version ]
               [ expression ]

DESCRIPTION
       Tcpdump  prints  out a description of the contents of packets on a net‐
       work interface that match the boolean expression;  the  description  is
       preceded  by a time stamp

You can run `tcpdump` directly, for example like in the following

```bash
$ sudo tcpdump
$ sudo tcpdump -i eth0 port 8080
```

One way to find all your network devices i to use `ifconfig`. For example, on your VM:

```bash
$ ifconfig
enp0s3    Link encap:Ethernet  HWaddr 08:00:27:19:2c:a4
          inet addr:10.0.2.15  Bcast:10.0.2.255  Mask:255.255.255.0
          inet6 addr: fe80::a00:27ff:fe19:2ca4/64 Scope:Link
          UP BROADCAST RUNNING MULTICAST  MTU:1500  Metric:1
          RX packets:17362 errors:0 dropped:0 overruns:0 frame:0
          TX packets:16945 errors:0 dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:1000
          RX bytes:1251926 (1.2 MB)  TX bytes:11818230 (11.8 MB)

lo        Link encap:Local Loopback
          inet addr:127.0.0.1  Mask:255.0.0.0
          inet6 addr: ::1/128 Scope:Host
          UP LOOPBACK RUNNING  MTU:65536  Metric:1
          RX packets:390 errors:0 dropped:0 overruns:0 frame:0
          TX packets:390 errors:0 dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:1
          RX bytes:213409 (213.4 KB)  TX bytes:213409 (213.4 KB)
```

On the VM, I invoke `tcpdump` like that:

```bash
sudo tcpdump -A -i enp0s9 -vv -X dst or src host 192.168.20.2 and port 8080
```

What do all the parameters mean? Check the man pages above or read one of the following articles:

  * https://danielmiessler.com/study/tcpdump/
  * https://www.cubrid.org/blog/understanding-tcp-ip-network-stack
  * http://www.skullbox.net/tcpudp.php


### Wireshark

In case you like it more visual, you might want to use WireShark https://www.wireshark.org instead of `tcpdump`.


--------------------------------------------


# Demo UCP server and Client in Go

## Server

```go
package main

import (
	"fmt"
	"net"
	"os"
	"time"
)

func main() {
	service := ":8080"
	udpAddr, err := net.ResolveUDPAddr("udp", service)
	checkError(err)
	conn, err := net.ListenUDP("udp", udpAddr)
	checkError(err)
	for {
		handleClient(conn)
	}
}

func handleClient(conn *net.UDPConn) {
	var buf [512]byte

	_, addr, err := conn.ReadFromUDP(buf[0:])
	fmt.Println(addr)
	if err != nil {
		return
	}
	daytime := time.Now().String()
	conn.WriteToUDP([]byte(daytime), addr)
}

func checkError(err error) {
	if err != nil {
		fmt.Fprintf(os.Stderr, "Fatal error: %s", err.Error())
		os.Exit(1)
	}
}
```

## Client

```go
package main

import (
	"fmt"
	"net"
	"os"
)

func main() {
	if len(os.Args) != 2 {
		fmt.Fprintf(os.Stderr, "Usage: %s host:port", os.Args[0])
		os.Exit(1)
	}
	service := os.Args[1]
	udpAddr, err := net.ResolveUDPAddr("udp", service)
	checkError(err)

	conn, err := net.DialUDP("udp", nil, udpAddr)
	defer conn.Close()
	checkError(err)
	_, err = conn.Write([]byte("anything"))
	checkError(err)
	var buf [512]byte
	n, err := conn.Read(buf[0:])
	checkError(err)
	fmt.Println(string(buf[0:n]))
	os.Exit(0)
}

func checkError(err error) {
	if err != nil {
		fmt.Fprintf(os.Stderr, "Fatal error: %s", err.Error())
		os.Exit(1)
	}
}
```

# Your turn!

![](images/your_turn.gif)

## First Hour

  * Create a small virtual network consisting of at least four nodes.
    - Let each node be either a virtual machines -decribed by a `Vagrantfile`- a Docker container -described by a `Dockerfile`in combination with a `docker-compose.yml` file.
    
  * Write a small TCP server in a language of your choice. Let it serve example data such as the current UTC time over a TCP connection when called.
  * Write a small TCP client in a language of your choice. This language might be different than the one used for your server. Let it call the TCP server and receive the example data via TCP packages.
  * Build your programs and deploy the server to one node and the client program to the remaining nodes.
  * Trigger the clients to call the server and observe the network traffic between your clients and server with either TCPDump or Wireshark
  * Capture a bit of network traffic and compare how packages are exchanged with the corresponding illustration in technique 24 from the book _Go in Practice_ by Matt Butcher Matt Farina, see Go_in_Practice_tech24.pdf.
  
## Second Hour

  * Write a small UDP server in a language of your choice. Let it serve example data such as the current UTC time over a UDP connection when called.
  * Write a small UDP client in a language of your choice. This language might be different than the one used for your server. Let it call the UDP server and receive the example data via UDP packages.
  * Build your programs and deploy the server to one node and the client program to the remaining nodes.
  * Trigger the clients to call the server and observe the network traffic between your clients and server with either TCPDump or Wireshark
  * Capture a bit of network traffic and compare how packages are exchanged with the corresponding illustration in technique 24 from the book _Go in Practice_ by Matt Butcher Matt Farina, see Go_in_Practice_tech24.pdf.



## Reflection

  * What is the difference between communication with TCP and UDP?
  * When would you choose which protocol?
  * How can these two protocols be us with respect to system integration?
  

## Third Hour

  * Extend the TCP server that you have written earlier, so that you implement a small protocoll. For example, let it serve the time only when it receives a message starting with `GET TIME` and let it serve a message containing `Hello, World!` when it receives a message starting with `SAY HI`.


# Hints

  * Start TCPDump on your server machine and let it inspect all ingoing and outgoing traffic to and from your server applications port.
  

# Further References

  * http://www.markhneedham.com/blog/2012/07/15/tcpdump-learning-how-to-read-udp-packets/
  ![](http://www.wtcs.org/snmp4tpc/images/TCP-Header.jpg)
  ![](http://www.wtcs.org/snmp4tpc/images/IP-Header.jpg)
  * http://www.wtcs.org/snmp4tpc/literature.htm    

  * https://www.itu.int/rec/dologin_pub.asp?lang=e&id=T-REC-X.200-199407-I!!PDF-E&type=items
  * https://support.microsoft.com/en-us/kb/103884
  * https://en.wikipedia.org/wiki/OSI_model

  * https://tools.ietf.org/html/rfc761
  * https://ipfs.io/ipfs/QmfYeDhGH9bZzihBUDEQbCbTc5k5FZKURMUoUvfmc27BwL/socket/udp_datagrams.html

  * https://systembash.com/a-simple-go-tcp-server-and-tcp-client/
  * https://stackoverflow.com/questions/38646224/golang-tcp-client-does-not-receive-data-from-server-hangs-blocks-on-conn-read